In [1]:
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string

from nltk.corpus import stopwords
import nltk
import numba
from numba import jit

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
!unzip english_dataset.zip
!rm -rf english_dataset.zip

Archive:  english_dataset.zip
   creating: english_dataset/
  inflating: __MACOSX/._english_dataset  
  inflating: english_dataset/english_dataset.tsv  
  inflating: __MACOSX/english_dataset/._english_dataset.tsv  
  inflating: english_dataset/hasoc2019_en_test-2919.tsv  
  inflating: __MACOSX/english_dataset/._hasoc2019_en_test-2919.tsv  


In [3]:
import pandas as pd
data_en=pd.read_csv('/content/english_dataset/english_dataset.tsv',delimiter='\t')
data_en.head(10)

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE
5,hasoc_en_6,@kellymiller513 @TheRealOJ32 I hope you rememb...,NOT,NONE,NONE
6,hasoc_en_7,@ICC Latest design of #WC2019 trophy. #CWC2019...,NOT,NONE,NONE
7,hasoc_en_8,#ADOS #trendingnow #blacklivesmatter #justice ...,HOF,PRFN,TIN
8,hasoc_en_9,Thanks for your support! Wow 600k. Graffiti ha...,NOT,NONE,NONE
9,hasoc_en_10,By wearing the #BalidaanBadge over his gloves ...,NOT,NONE,NONE


**CONVERTING ALL LETTERS TO LOWERCASE OR UPPERCASE**

In [4]:
data_lower=data_en
data_lower["text"].str.lower()

0       #dhonikeepstheglove | watch: sports minister k...
1       @politico no. we should remember very clearly ...
2       @cricketworldcup guess who would be the winner...
3       corbyn is too politically intellectual for #bo...
4       all the best to #teamindia for another swimmin...
                              ...                        
5847    @davidfrum @trueblueusa1 that's cute and all, ...
5848    a recession issa comin' #maga #magamyass #fuck...
5849    #doctorsfightback  will 'the mad n irrational ...
5850    #shiningindia #educatedindia or more like rund...
5851    could this be our new prime minister?     #ric...
Name: text, Length: 5852, dtype: object

**Removing numbers**

In [5]:
english_en=pd.read_csv('/content/english_dataset/english_dataset.tsv',delimiter='\t')
english_num=english_en
english_num['text']=english_num['text'].str.replace('\d+',' ')
english_en.head(10)

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE
5,hasoc_en_6,@kellymiller @TheRealOJ I hope you remembere...,NOT,NONE,NONE
6,hasoc_en_7,@ICC Latest design of #WC trophy. #CWC #CWC ...,NOT,NONE,NONE
7,hasoc_en_8,#ADOS #trendingnow #blacklivesmatter #justice ...,HOF,PRFN,TIN
8,hasoc_en_9,Thanks for your support! Wow k. Graffiti has ...,NOT,NONE,NONE
9,hasoc_en_10,By wearing the #BalidaanBadge over his gloves ...,NOT,NONE,NONE


**Removing punctuations**

In [ ]:
english_en=pd.read_csv('/content/english_dataset/english_dataset.tsv',delimiter='\t')
puntuations='''''!()-[]{};:'"\,<>./?@#$%^&*_~'''
english_punc=english_en
english_punc['text']=english_punc['text'].str.replace('[^\w\s]','')
english_en.head(15)

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,DhoniKeepsTheGlove WATCH Sports Minister Kire...,NOT,NONE,NONE
1,hasoc_en_2,politico No We should remember very clearly th...,HOF,HATE,TIN
2,hasoc_en_3,cricketworldcup Guess who would be the winner ...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for Bor...,NOT,NONE,NONE
4,hasoc_en_5,All the best to TeamIndia for another swimming...,NOT,NONE,NONE
5,hasoc_en_6,kellymiller513 TheRealOJ32 I hope you remember...,NOT,NONE,NONE
6,hasoc_en_7,ICC Latest design of WC2019 trophy CWC2019 CWC...,NOT,NONE,NONE
7,hasoc_en_8,ADOS trendingnow blacklivesmatter justice fuck...,HOF,PRFN,TIN
8,hasoc_en_9,Thanks for your support Wow 600k Graffiti has ...,NOT,NONE,NONE
9,hasoc_en_10,By wearing the BalidaanBadge over his gloves m...,NOT,NONE,NONE


**Removing Stopwords**

In [7]:
from nltk.tokenize import word_tokenize
english_en=pd.read_csv('/content/english_dataset/english_dataset.tsv',delimiter='\t')
stop=stopwords.words('english')
english_en['text']=english_en['text'].apply(word_tokenize)
english_en['text']=english_en['text'].apply(lambda x:[item for item in x if item not in stop])
english_en['text'].head(10)


0    [#, DhoniKeepsTheGlove, |, WATCH, :, Sports, M...
1    [@, politico, No, ., We, remember, clearly, #,...
2    [@, cricketworldcup, Guess, would, winner, #, ...
3    [Corbyn, politically, intellectual, #, BorisJo...
4    [All, best, #, TeamIndia, another, swimming, c...
5    [@, kellymiller513, @, TheRealOJ32, I, hope, r...
6    [@, ICC, Latest, design, #, WC2019, trophy, .,...
7    [#, ADOS, #, trendingnow, #, blacklivesmatter,...
8    [Thanks, support, !, Wow, 600k, ., Graffiti, m...
9    [By, wearing, #, BalidaanBadge, gloves, @, msd...
Name: text, dtype: object

**Removing white spaces**

In [8]:
english_en=pd.read_csv('/content/english_dataset/english_dataset.tsv',delimiter='\t')
english_white=english_en
english_white['text']=english_white['text'].str.strip()
english_white['text']

0       #DhoniKeepsTheGlove | WATCH: Sports Minister K...
1       @politico No. We should remember very clearly ...
2       @cricketworldcup Guess who would be the winner...
3       Corbyn is too politically intellectual for #Bo...
4       All the best to #TeamIndia for another swimmin...
                              ...                        
5847    @davidfrum @trueblueusa1 That's cute and all, ...
5848    a recession issa comin' #maga #magamyass #fuck...
5849    #DoctorsFightBack  Will 'The Mad n Irrational ...
5850    #ShiningIndia #educatedindia or more like RUND...
5851    Could this be our new Prime Minister?     #Ric...
Name: text, Length: 5852, dtype: object